# Meteo idiote

cf README.md

## Analyser les données

In [1]:
import numpy as np
from numpy import genfromtxt
from numpy import hsplit
my_data = genfromtxt('iadata.csv', delimiter=';', dtype="f4,u4,S8", names=True)
(temp,hour,label)=my_data['t'],my_data['h'],my_data['season']
print ("temp",temp)
print ("hour",hour)

temp [278.55 277.35 276.85 ... 281.05 282.35 282.55]
hour [     0      0      0 ... 210000 210000 210000]


In [2]:
import matplotlib.pyplot as plt
my_rec = my_data.view(np.recarray)
plt.plot(my_rec.h,my_rec.t,'b.')
#plt.ylim (0.,210000)
plt.show()

plt.plot(my_rec.season,my_rec.t,'b.')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Les données montrent que le modèle ne sera pas pertinent entre 275°K (2°c) et 295°C (22°C).
On s'en doutait...

## Préparation des données d'entrées

In [3]:
from keras import utils
print('Temperature en K : ',temp)
#temp_normalized = utils.normalize(temp)
temp_normalized = temp/350
print('Temperature normalisee : ', temp_normalized)
print('Heure en String : ',hour)
hour_normalized = hour/240000
print('Heure normalisee : ', hour_normalized)


Using TensorFlow backend.


Temperature en K :  [278.55 277.35 276.85 ... 281.05 282.35 282.55]
Temperature normalisee :  [0.79585713 0.7924286  0.791      ... 0.803      0.8067143  0.80728567]
Heure en String :  [     0      0      0 ... 210000 210000 210000]
Heure normalisee :  [0.    0.    0.    ... 0.875 0.875 0.875]


In [4]:
from keras import models
from keras import layers

from numpy import stack
feature=stack((temp_normalized,hour_normalized),axis=1)
print(feature)

feature = feature.reshape((161449,2))
feature = feature.astype('float32')

print(feature.ndim)

[[0.79585713 0.        ]
 [0.79242861 0.        ]
 [0.79100001 0.        ]
 ...
 [0.80299997 0.875     ]
 [0.8067143  0.875     ]
 [0.80728567 0.875     ]]
2


In [5]:
feature[6819]

array([0.819, 0.   ], dtype=float32)

In [6]:
from numpy import average
average(feature.T[0])

0.8126421

## Transformer les variables de sortie (les saisons) en vecteur de probabilité de saison

In [7]:
label

array([b'winter', b'winter', b'winter', ..., b'winter', b'winter',
       b'winter'], dtype='|S8')

In [8]:
label.ndim

1

In [9]:
len(label)
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transformed_label = encoder.fit_transform(label)
print(transformed_label)

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 ...
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [10]:
#from keras.preprocessing.text import Tokenizer
#tokenizer = Tokenizer(char_level=True)
#tokenizer.fit_on_texts(label)
#sequence_of_int = tokenizer.texts_to_sequences(label)
#print (sequence_of_int)
#from keras.utils import to_categorical
#new_label= to_categorical(label, num_classes=4, dtype='int32')
#new_label= to_categorical(label, num_classes=4)
#print (new_label)

In [11]:

transformed_label[6819]

array([0, 0, 1, 0])

## Construction du réseau de neurone et compilation

In [12]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu',input_dim = 2))
network.add(layers.Dense(4, activation='softmax'))
#definition de la fonction de perte et de l'algo d'optim
network.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])


## Entrainement et évaluation de l'entrainement

TODO : séparer les variables d'entrainbement et de contrôle du modèle (il doit y avoir une fonction pour ça)

In [13]:
network.fit(feature, transformed_label,epochs=5, batch_size=128)

test_loss, test_acc = network.evaluate(feature, transformed_label)
print ('test_acc:', test_acc)

Epoch 1/5
161449/161449 [==============================] - 1s 9us/step - loss: 1.3707 - acc: 0.3121
Epoch 2/5
161449/161449 [==============================] - 1s 7us/step - loss: 1.2684 - acc: 0.4474
Epoch 3/5
161449/161449 [==============================] - 1s 8us/step - loss: 1.1642 - acc: 0.4791
Epoch 4/5
161449/161449 [==============================] - 1s 7us/step - loss: 1.1128 - acc: 0.4869
Epoch 5/5
161449/161449 [==============================] - 1s 9us/step
test_acc: 0.48962830367484467


## Prédiction d'été à midi 32°C

In [14]:
prediction = network.predict(np.array([[(32.+273.15)/350., 120000./240000.]]))
print(prediction)
predicted_label = encoder.inverse_transform(prediction)
print(predicted_label)

[[7.5496850e-03 5.0528873e-02 9.4169956e-01 2.2178818e-04]]
[b'summer']


## Prédiction d'hiver à minuit -5°C

In [15]:
prediction = network.predict(np.array([[(273.15-5)/350., (0./240000.)]]))
print(prediction)
predicted_label = encoder.inverse_transform(prediction)
print(predicted_label)

[[0.2237666  0.0657198  0.00622501 0.7042886 ]]
[b'winter']


In [16]:
from keras.models import model_from_json
import os

In [17]:
# serialize model to JSON
model_json = network.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
#network.save_weights("model.h5")
network.save("model.h5")
# save to tensorflowjs
#import tensorflowjs as tfjs
#tfjs.converters.save_keras_model(network)
print("Saved model to disk")


Saved model to disk


In [18]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [19]:
Temperature=10.
Heure=30000.

In [20]:
prediction = loaded_model.predict(np.array([[(273.15+Temperature)/350, Heure/240000.]]))
print("Les probas sont" ,prediction)
predicted_label = encoder.inverse_transform(prediction)
print("La prédiction est " ,predicted_label)

Les probas sont [[0.2842217  0.31012064 0.27595216 0.12970549]]
La prédiction est  [b'spring']


## The end